<a href="https://colab.research.google.com/github/Serendipity-Song/Classify_CIFAR-10_Dataset/blob/main/CIFAR_10%2C_resnet56.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu', batch_normalization=True):
    x = layers.Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer='he_normal')(inputs)
    if batch_normalization:
        x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    return x
def resnet_block(inputs, num_filters=16, num_res_blocks=3):
    x = inputs
    for _ in range(num_res_blocks):
        y = resnet_layer(x, num_filters=num_filters)
        y = resnet_layer(y, num_filters=num_filters, activation=None)
        x = layers.add([x, y])
        x = layers.Activation('relu')(x)
    return x

In [ ]:
def resnet56(input_shape, num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    x = resnet_layer(inputs)

    # Instantiate the stack of residual units
    x = resnet_block(x, num_filters=16, num_res_blocks=9) # 18 layers
    x = resnet_layer(x, num_filters=32, strides=2)
    x = resnet_block(x, num_filters=32, num_res_blocks=9) # 18 layers
    x = resnet_layer(x, num_filters=64, strides=2)
    x = resnet_block(x, num_filters=64, num_res_blocks=9) # 18 layers

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
# Load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Create an instance of the model
model = resnet56(input_shape=x_train.shape[1:])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

170498071/170498071 [==============================] - 12s 0us/step
Epoch 1/10
1563/1563 [==============================] - 2238s 1s/step - loss: 1.5095 - accuracy: 0.4484 - val_loss: 1.6229 - val_accuracy: 0.4792
Epoch 2/10
1563/1563 [==============================] - 2185s 1s/step - loss: 0.9984 - accuracy: 0.6444 - val_loss: 1.9466 - val_accuracy: 0.4769
Epoch 3/10
1563/1563 [==============================] - 2118s 1s/step - loss: 0.7800 - accuracy: 0.7275 - val_loss: 0.9623 - val_accuracy: 0.6838
Epoch 4/10
1563/1563 [==============================] - 2076s 1s/step - loss: 0.6493 - accuracy: 0.7737 - val_loss: 0.8621 - val_accuracy: 0.7067
Epoch 5/10
1563/1563 [==============================] - 1994s 1s/step - loss: 0.5635 - accuracy: 0.8045 - val_loss: 0.7114 - val_accuracy: 0.7667
Epoch 6/10
 140/1563 [=>............................] - ETA: 28:51 - loss: 0.4714 - accuracy: 0.8364

In [ ]:
resnet56.save('/content/drive/MyDrive/1.DeepArc')

AttributeError: ignored